# Implémentez un modèle de scoring

- **Projet 7 du parcours « Data Scientist » d’OpenClassrooms**
- **Mark Creasey**

## Étape 1 : Préparation du jeu de données et feature engineering


## 1.1 Compréhension du problème

### 1.1.1 Problématique

La société financière, nommée **"Prêt à dépenser"**, propose des crédits à la consommation pour des
personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre **un outil de “scoring crédit”** pour calculer la qu’un client
rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc
développer **un algorithme de classification** en s’appuyant sur des sources de données variées (données
comportementales, données provenant d'autres institutions financières, etc.).

### 1.1.2 Les données

Voici [les données](https://www.kaggle.com/c/home-credit-default-risk/data) pour réaliser le
dashboard. Pour plus de simplicité, vous pouvez les télécharger à
[cette adresse](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip).

### 1.1.1 Mission

- Sélectionner un kernel Kaggle pour faciliter la préparation des données nécessaires à l’élaboration du modèle de scoring.
- Analyser ce kernel et l’adapter aux besoins de votre mission.

Focalise sur :

1. La construction d'un **modèle de scoring** qui donnera une prédiction sur la probabilité de faillite
   d'un client de façon automatique.
   - élaboration
   - optimisation
   - comprehension (interpretabilité)
   
2. Construction d'un **dashboard interactif** qui montre avec transparence les décisions d’octroi de
   crédit, à destination des gestionnaires de la relation client permettant d'interpréter les
   prédictions faites par le modèle et d’améliorer la connaissance client des chargés de relation
   client.


## 1.2 Definition de l'environnement

- `local`  : Développement local (avec échantillon de 50 Mo de données)
- `colab`  : Google Colab
- `kaggle` : Kaggle Kernel

In [1]:
ENV='local'

if ENV=='local':
    # local development
    DATA_FOLDER = '../data/raw'
    OUT_FOLDER = '../data/out'
    IMAGE_FOLDER = '../images/nettoyage'

if ENV == 'colab':
    # Colaboratory - uncomment les 2 lignes suivant pour connecter à votre drive
    # from google.colab import drive
    # drive.mount('/content/drive')
    DATA_FOLDER = '/content/drive/MyDrive/data/OC7'
    OUT_FOLDER = '/content/drive/MyDrive/data/OC7'
    IMAGE_FOLDER = '/content/drive/MyDrive/images/OC7/nettoyage'



## 1.3 Fichiers de données

1. Les données en format CSV (>700Mb compactés) sont à télecharger de ce lien: 
- https://www.kaggle.com/c/home-credit-default-risk/data
- Pour plus de simplicité, vous pouvez les télécharger à [cette adresse.](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip)

2.  Placer le fichier compacté (**.zip**) dans le **DATA_FOLDER** défini ci-dessous

### Noms des fichiers de données (identique pour nettoyage et l'analyse exploratoire)

- Le grand fichier zip des données doit être placé dans `DATA_FOLDER` au préalable 
- Tous les autres fichiers de données sont téléchargés ou crées pendant le nettoyage, puis enregistrés dans `OUT_FOLDER`

In [2]:
# Données (DATA_FOLDER)
ZIPPED_DATA_FILENAME=f'Projet+Mise+en+prod+-+home-credit-default-risk.zip'
RAW_DATA_FILENAME='HomeCredit_columns_description.csv'
SAMPLE_DATA_FILENAME='HomeCredit_columns_description.csv'


# Données nettoyés (OUT_FOLDER)
CLEAN_DATA_FILENAME='cleaned_data_scoring.csv'
CLEAN_DATA_SAMPLE='cleaned_data_sample.csv' # 100,000 registres
SAMPLE_SIZE=100000

## 1.4 Requirements: Bibliothèques utilisées dans ce notebook
Ce notebook marche a été testé en developpement local, sur Google Colab et Kaggle

```txt
# copy dans un fichier requirements.txt, puis
# !pip install -r requirements.txt
python>=3.7,<=3.9
numpy>=1.19.5,<=1.21.2
pandas>=1.1.5,<=1.3.4 
matplotlib>=3.2.2,<=3.5.0
seaborn==0.11.2
scikit-learn>=1.0.1
```

In [3]:
# Decommentarise la ligne suivant si vous ne voulez pas changer vos versions existants
# !pip install numpy pandas matplotlib seaborn scipy sklearn missingno requests

## 1.5 Import dependencies

### 1.5.1 Import des bibliothèques utilisées par ce notebook

In [4]:
# suppress furture warnings de pandas 1.3.0  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import gc
import time
from contextlib import contextmanager


### 1.5.2 Liste des versions des bibliothèques utilisées

In [5]:
!python --version
print('versions des bibliothèques utilisées:')
print('; '.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

Python 3.7.0
versions des bibliothèques utilisées:
numpy==1.21.5; pandas==1.1.5; seaborn==0.11.2; sklearn==1.0.2


### 1.5.3 Configuration défauts d'affichage

In [6]:
pd.set_option('display.max_columns', 200)  # pour afficher toutes les colonnes
pd.set_option('display.max_rows', 10)  # pour afficher max 10 lignes
pd.set_option('display.max_colwidth', 100)

%matplotlib inline
sns.set_theme(style="white", context="notebook")
sns.set_color_codes("pastel")
sns.set_palette("tab20")

## 1.6 Fonctions utilitaires (make_dirs, savefig)
- pour enregistrer les graphiques, define **`SAVE_IMAGES = True`**


In [7]:
def os_make_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

def os_path_join(folder, file):
    """remplacement pour `os.path.join(folder, file)` sur windows"""
    return f'{folder}/{file}'

if ENV !='kaggle': 
    os_make_dir(DATA_FOLDER)
    os_make_dir(OUT_FOLDER)

os_make_dir(IMAGE_FOLDER)
SAVE_IMAGES=True
def to_png(fig_name):
    if SAVE_IMAGES:
        """Enregistre l'image"""
        plt.gcf().savefig(os_path_join(IMAGE_FOLDER,f'{fig_name}.png'), bbox_inches="tight")

# 2. Importation des données <a name="importation"></a>

## 2.1 Configuration de l'environnement de travail

### 2.1.1 Installation des bibliothèques nécessaires pour manipuler les données

Pour plusieurs notebooks dans un dossier, enregistre une liste des bibliothèques dans un fichier `requirements.txt`

In [8]:
def install_libraries(required=None) -> None:
    """
    Installer les bibliothèques nécessaires pour ce notebook
    - https://stackoverflow.com/questions/44210656/how-to-check-if-a-module-is-installed-in-python-and-if-not-install-it
    """
    if required is None:
        required = {'numpy', 'pandas', 'matplotlib', 'seaborn'}
    import sys, subprocess, pkg_resources
    installed = {pkg.key for pkg in pkg_resources.working_set}
    missing = set(required) - set(installed)
    print(f'required modules: {list(required)}')
    print(f'missing modules: {list(missing)}')
    if missing:
        python = sys.executable
        subprocess.check_call([python, '-m', 'pip', 'install', *missing],
                              stdout=subprocess.DEVNULL)


install_libraries({'numpy', 'pandas', 'matplotlib', 'seaborn', 'requests', 'missingno'})

required modules: ['missingno', 'matplotlib', 'numpy', 'requests', 'seaborn', 'pandas']
missing modules: []


## 2.2 Configuration de l'importation des données

### 2.2.1 Choix de fichier à analyser

In [9]:
DATA_FILENAME = SAMPLE_DATA_FILENAME if ENV=='local' else RAW_DATA_FILENAME
RAW_DATA = os_path_join(DATA_FOLDER, DATA_FILENAME)
DATA_ZIPPED = os_path_join(DATA_FOLDER,ZIPPED_DATA_FILENAME)
print(f'data file: {RAW_DATA}')

data file: ../data/raw/HomeCredit_columns_description.csv


In [10]:
def unzip_data_si_besoin(env=ENV):
    """procedure pour unzip sur Google Drive via Google Colab"""
    if os.path.exists(RAW_DATA):
        print(f'data CSV file exists ({RAW_DATA})')
    else:
        print(f'data CSV file does not exist ({RAW_DATA})')
        if env=='colab' and os.path.exists(DATA_ZIPPED):
            # uncomment les 3 lignes suivants
            print (f'unzipping {DATA_ZIPPED}')
            !unzip {DATA_ZIPPED} -d {DATA_FOLDER}
            print (f'{DATA_ZIPPED} has been unzipped')
            if os.path.exists(RAW_DATA):
                print(f'data CSV file now exists ({RAW_DATA})')
        else:
            print(f'zipped data does not exist ({DATA_ZIPPED})')

unzip_data_si_besoin(ENV)

data CSV file exists (../data/raw/HomeCredit_columns_description.csv)


### 2.2.2 Information sur le fichier (taille, type, nb. registres, champs)

In [11]:
def get_size_str(octets):
    gb = round(octets / 2 ** 30, 2)
    mb = round(octets / 2 ** 20, 2)
    kb = round(octets / 2 ** 10, 2)
    if gb > 1:
        ret = f'{gb} Go'
    elif mb > 1:
        ret = f'{mb} Mo'
    elif kb > 1:
        ret = f'{kb} ko'
    else:
        ret = f'{octets} octets'
    return ret

def get_filesize(file_path=RAW_DATA):
    """Taille du fichier"""
    octets = os.stat(file_path).st_size
    return get_size_str(octets)

for filename in os.listdir(DATA_FOLDER):
    if filename.lower().endswith('csv'):
        filepath=f'{DATA_FOLDER}/{filename}'
        print(f'file : {filename} : size={get_filesize(filepath)}')


file : application_test.csv : size=25.34 Mo
file : application_train.csv : size=158.44 Mo
file : bureau.csv : size=162.14 Mo
file : bureau_balance.csv : size=358.19 Mo
file : credit_card_balance.csv : size=404.91 Mo
file : HomeCredit_columns_description.csv : size=36.51 ko
file : installments_payments.csv : size=689.62 Mo
file : POS_CASH_balance.csv : size=374.51 Mo
file : previous_application.csv : size=386.21 Mo
file : sample_submission.csv : size=523.63 ko


## 2.3 Data schema

Le schema et description des données est fourni sur le lien:
- https://www.kaggle.com/competitions/home-credit-default-risk/data

<p><img title="" alt="Data" src="img/home_credit.png"></p>
<em>Original source : https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png<em>

### 2.3.1 Description des champs
- details sur <https://www.kaggle.com/competitions/home-credit-default-risk/data>

In [12]:
def field_descriptions(data_dir=DATA_FOLDER, num_rows=1000):
    df = pd.read_csv(f'{data_dir}/HomeCredit_columns_description.csv', nrows= num_rows, encoding= 'unicode_escape')
    return df


field_descriptions().head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment difficulties: he/she had late payment more than X days ...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
